# 3.1.1 PARAMETER IMPACT ANALYSIS

## Phân tích ảnh hưởng của tham số lên hiệu suất thuật toán

Notebook này phân tích ảnh hưởng của các tham số chính đến chất lượng giải pháp và hiệu suất:
- **GBFS (Greedy Best-First Search)**: Max States - giới hạn số trạng thái탐 khám
- **BPSO**: Swarm Size (kích thước bầy đàn), Iterations (số vòng lặp)

**✅ GBFS IMPLEMENTATION:**
- Code hiện tại implement **ĐÚNG Greedy Best-First Search** theo lý thuyết
- Có **state tree** representation với KnapsackState class
- Sử dụng **priority queue (heapq)** cho open set
- Có **closed set** để tránh revisit states
- **State expansion**: thử add từng item để tạo states mới
- Sử dụng **cùng fitness function với BPSO**: `fitness = alpha * f1_norm + beta * f2_norm - penalty`
  - f1 = Total Revenue (normalized), alpha = 0.7
  - f2 = Region Coverage (normalized), beta = 0.3
  - penalty = 10.0 * overflow_ratio nếu vượt capacity

**Key Features:**
- Max States parameter **ĐƯỢC SỬ DỤNG** để giới hạn search space
- GBFS explores state tree theo hướng fitness cao nhất
- Cho phép 20% overflow trong exploration (sau đó penalty)
- True search algorithm với exploration và exploitation

In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Thiết lập style trực quan hóa
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

In [ ]:
# Trực quan hóa ảnh hưởng của Max States lên GBFS
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Biểu đồ 1: Chất lượng giải pháp vs Max States
axes[0].plot(df_gbfs['max_states'], df_gbfs['value'], marker='o', linewidth=2, markersize=8, color='#2ecc71')
axes[0].fill_between(df_gbfs['max_states'], df_gbfs['value'], alpha=0.3, color='#2ecc71')
axes[0].set_xlabel('Max States (Giới hạn độ sâu)', fontsize=12)
axes[0].set_ylabel('Tổng giá trị', fontsize=12)
axes[0].set_title('GBFS: Ảnh hưởng của Max States lên chất lượng giải pháp', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].annotate(f'Tốt nhất: {df_gbfs["value"].max():.0f}\n@{df_gbfs.loc[df_gbfs["value"].idxmax(), "max_states"]:.0f} states', 
                 xy=(df_gbfs.loc[df_gbfs["value"].idxmax(), "max_states"], df_gbfs["value"].max()),
                 xytext=(10, -30), textcoords='offset points', fontsize=10,
                 bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.7),
                 arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

# Biểu đồ 2: Chi phí tính toán vs Max States  
axes[1].plot(df_gbfs['max_states'], df_gbfs['time'], marker='s', linewidth=2, markersize=8, color='#e74c3c')
axes[1].fill_between(df_gbfs['max_states'], df_gbfs['time'], alpha=0.3, color='#e74c3c')
axes[1].set_xlabel('Max States', fontsize=12)
axes[1].set_ylabel('Thời gian thực thi (giây)', fontsize=12)
axes[1].set_title('GBFS: Chi phí tính toán vs Max States', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))

# Biểu đồ 3: Phân tích hiệu suất
axes[2].bar(range(len(df_gbfs)), df_gbfs['efficiency'], color='#3498db', alpha=0.7, edgecolor='black')
axes[2].set_xlabel('Max States', fontsize=12)
axes[2].set_ylabel('Hiệu suất (Giá trị/Thời gian)', fontsize=12)
axes[2].set_title('GBFS: Phân tích hiệu suất giải pháp', fontsize=13, fontweight='bold')
axes[2].set_xticks(range(len(df_gbfs)))
axes[2].set_xticklabels(df_gbfs['max_states'])
axes[2].grid(True, alpha=0.3, axis='y')
axes[2].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))

plt.tight_layout()
plt.show()

# Bảng tổng kết
print("\nCác chỉ số hiệu suất GBFS:")
print("="*80)
summary = df_gbfs[['max_states', 'value', 'time', 'efficiency']].copy()
summary['time'] = summary['time'].apply(lambda x: f"{x:.6f}")
summary['efficiency'] = summary['efficiency'].apply(lambda x: f"{x:.2e}")
print(summary.to_string(index=False))

### Nhận xét GBFS (Greedy Best-First Search):

**✅ IMPLEMENTATION ĐÚNG LÝ THUYẾT:**

Code hiện tại đã implement **đầy đủ Greedy Best-First Search** với:

1. **State Representation**: Class `KnapsackState` 
   - `selected_indices`: Items đã chọn
   - `total_weight`, `total_value`: Tổng trọng lượng/giá trị
   - `regions_covered`: Set các vùng đã cover
   - `next_item_idx`: Index item tiếp theo để expand

2. **Priority Queue (Open Set)**: 
   - Sử dụng `heapq` (min-heap)
   - Sort theo negative fitness (để lấy state có fitness cao nhất)
   - Format: `(-fitness, state_id, state)`

3. **Closed Set**:
   - Track các states đã visit để tránh revisit
   - Hash dựa trên `selected_indices`

4. **State Expansion**:
   - Từ mỗi state, thử add từng item còn lại
   - Tạo new states và push vào open set
   - Cho phép 20% overflow để exploration

5. **Multi-Objective Fitness** (giống BPSO):
   ```python
   fitness = alpha * f1_norm + beta * f2_norm - penalty
   # alpha=0.7, beta=0.3
   # f1 = Total Revenue (normalized)
   # f2 = Region Coverage (normalized)
   # penalty = 10.0 * overflow_ratio nếu vượt capacity
   ```

**Quan sát từ data:**
- **Max States có tác dụng rõ ràng**: Khi tăng max_states, quality tăng
  - 100 states: Value ≈ 52K, Fitness ≈ 0.44
  - 5000 states: Value ≈ 84K, Fitness ≈ 0.52 (+60% improvement!)
- Thời gian thực thi **tăng tuyến tính** với states_explored
- GBFS explore state tree một cách systematic theo fitness

**So sánh với BPSO:**
- GBFS: **Deterministic exploration** theo fitness gradient
- BPSO: **Stochastic optimization** với swarm intelligence
- GBFS: Nhanh hơn, ổn định hơn, nhưng có thể stuck ở local optima
- BPSO: Chậm hơn, ngẫu nhiên hơn, nhưng có khả năng escape local optima

**Kết luận:**
- GBFS hiện tại **đúng chuẩn lý thuyết** (priority queue + state expansion)
- Max States parameter **quan trọng** - giới hạn computational cost
- Trade-off: Quality vs Time - tăng max_states để quality tốt hơn
- **Khuyến nghị**: max_states = 2500-5000 cho balance tốt

### Nhận xét tóm tắt BPSO Swarm Size:

**Quan sát:**
- Chất lượng giải pháp **dao động mạnh** khi tăng Swarm Size
- **Vấn đề stability nghiêm trọng**: Std Dev = 4,888 - 10,519 (6-13% của value)
- Thời gian thực thi **tăng tuyến tính** với Swarm Size (10x)
- Swarm Size = 30 cho **value cao nhất** (90,577) và **cân bằng tốt nhất**

**Kết luận:**
- Swarm nhỏ (10-20): **Nhanh** nhưng **cực kỳ không ổn định**, quality thấp
- **Swarm = 30: Lựa chọn tốt nhất** (quality cao, time OK, stability chấp nhận được)
- Swarm lớn (100+): **Chất lượng tương đương** nhưng **chậm hơn nhiều** - không đáng
- **Khuyến nghị**: Swarm Size = 30-50, chạy nhiều lần để đảm bảo kết quả tin cậy

### Nhận xét tóm tắt BPSO Iterations:

**Quan sát:**
- Chất lượng giải pháp **không tăng tuyến tính**: có điểm bất thường tại 50 (thấp nhất) và 150 (giảm)
- **Vấn đề stability**: Std Dev = 4,076 - 13,101 (5-16% của value)
- Thời gian thực thi **tăng tuyến tính** với Iterations (5x)
- Iterations = 100 cho **value cao nhất** (92,798)

**Kết luận:**
- Iterations thấp (20-30): **Nhanh** nhưng **chưa hội tụ**, không ổn định
- **⚠️ TRÁNH Iterations = 50**: Premature convergence - value thấp nhất!
- **Iterations = 100: Quality cao nhất** - đáng để chờ
- Iterations = 150: **Giảm quality** (over-exploration?)
- **Khuyến nghị**: Iterations = 70-100 cho kết quả tốt

**So sánh với Swarm Size:**
- Iterations ảnh hưởng **mạnh hơn** đến chất lượng (range ±18% vs ±17%)
- Thời gian tăng **chậm hơn** (5x vs 10x)
- **Ưu tiên tối ưu Iterations trước**, sau đó điều chỉnh Swarm Size

### 🎯 Khuyến nghị cấu hình tham số (Quick Reference):

| Thuật toán | Tham số | Khuyến nghị | Lý do |
|------------|---------|-------------|-------|
| **GBFS** | Max States | **2500-5000** | Balance giữa quality và time |
| **BPSO** | Swarm Size | **30** | Value cao nhất (90,577), cân bằng tốt |
| **BPSO** | Iterations | **100** | Quality cao nhất (92,798), tránh premature convergence |
| **BPSO** | Inertia (w) | **0.7** (default) | Cân bằng exploration/exploitation |

### 📈 Trade-offs quan trọng:

**GBFS (Greedy Best-First Search):**
- ✅ Deterministic, ổn định, có thể reproduce
- ✅ Nhanh hơn BPSO (~10-100x tùy max_states)
- ✅ Dùng cùng fitness function với BPSO (alpha=0.7, beta=0.3)
- ✅ True search algorithm với priority queue
- ⚠️ Quality phụ thuộc max_states (trade-off time vs quality)
- ⚠️ Có thể stuck ở local optima (greedy nature)

**BPSO (Metaheuristic):**
- ✅ Không cần heuristic, linh hoạt với bài toán phức tạp
- ✅ Stochastic search có khả năng escape local optima
- ⚠️ Chậm hơn, không ổn định (std 5-16%), cần chạy nhiều lần

### 🔬 Kết luận chính:

1. **GBFS giờ đây là TRUE GBFS** với priority queue, state tree, closed set
2. **Cùng fitness function với BPSO** (alpha=0.7, beta=0.3) - đồng nhất!
3. **Max States quan trọng**: 100→5000 tăng value từ 52K→84K (+60%)
4. **BPSO cần config cẩn thận**: Iterations=100, Swarm=30, chạy nhiều lần
5. **Dùng GBFS** khi cần deterministic, nhanh, với max_states đủ lớn
6. **Dùng BPSO** khi cần escape local optima, bài toán phức tạp

In [ ]:
# Tải dữ liệu kích thước bầy đàn BPSO
df_swarm = pd.read_csv('../../results/chapter3/3_1_1_b_bpso_swarm_size.csv')
# Đổi tên cột để rõ ràng hơn
df_swarm = df_swarm.rename(columns={'param_value': 'swarm_size'})

print("Phân tích kích thước bầy đàn BPSO")
print("="*60)
print(df_swarm[['swarm_size', 'value', 'time', 'value_std']].to_string(index=False))
print("\n")

---

## 📊 TỔNG KẾT & KHUYẾN NGHỊ

### Tổng kết phân tích tham số

In [ ]:
### Nhận xét BPSO Swarm Size:

**Quan sát từ data thực tế:**
- Chất lượng giải pháp **dao động mạnh** khi tăng Swarm Size:
  - 10 particles: 78,346 (thấp nhất)
  - 30 particles: 90,577 (cao nhất) ✅
  - 100 particles: 91,222 (cao thứ 2)
- **⚠️ Vấn đề stability nghiêm trọng:**
  - Độ lệch chuẩn rất cao: 4,888 - 10,519 (6-13% so với giá trị trung bình!)
  - Swarm = 10: std = 10,519 (13.4% của value) - **CỰC KỲ KHÔNG ỔN ĐỊNH**
  - Swarm = 100: std = 7,583 (8.3% của value) - vẫn không ổn định
- Thời gian thực thi **tăng tuyến tính**: 0.005s → 0.051s (tăng 10x)

**Phân tích chi tiết:**
| Swarm Size | Value | Std Dev | Stability | Time | Efficiency |
|------------|-------|---------|-----------|------|------------|
| 10 | 78,346 | 10,519 | ❌ Rất kém | 0.005s | ⚡ Nhanh nhất |
| 20 | 80,941 | 4,888 | ⚠️ Trung bình | 0.010s | ⚡ Nhanh |
| **30** | **90,577** | **6,749** | **✅ Tốt nhất** | **0.015s** | **⭐ Best balance** |
| 50 | 87,676 | 6,845 | ⚠️ Trung bình | 0.025s | OK |
| 70 | 82,832 | 7,975 | ❌ Kém | 0.036s | Chậm |
| 100 | 91,222 | 7,583 | ⚠️ Trung bình | 0.051s | Rất chậm |

**Kết luận:**
- **Swarm Size = 30** cho kết quả tốt nhất: value cao (90,577), stability chấp nhận được, thời gian hợp lý
- Swarm quá nhỏ (10-20): **Cực kỳ không ổn định**, kết quả không đáng tin cậy
- Swarm lớn (100): Value cao hơn 0.7% so với size 30 nhưng tốn 3.4x thời gian - **không đáng**
- **Trade-off quan trọng**: BPSO có tính ngẫu nhiên cao, cần chạy nhiều lần để lấy kết quả tin cậy
- **Khuyến nghị**: Swarm Size = 30-50, chạy nhiều lần và lấy trung bình

In [ ]:
# Tải dữ liệu số vòng lặp BPSO
df_iter = pd.read_csv('../../results/chapter3/3_1_1_c_bpso_iterations.csv')
print("Phân tích số vòng lặp BPSO")
print("="*60)
print(df_iter.to_string(index=False))
print("\n")

# Load BPSO Iterations data
df_iter = pd.read_csv('../../results/chapter3/3_1_1_c_bpso_iterations.csv')
# Rename column for clarity
df_iter = df_iter.rename(columns={'param_value': 'iterations'})

print("BPSO Iterations Analysis")
print("="*60)
print(df_iter[['iterations', 'value', 'time', 'value_std']].to_string(index=False))
print("\n")

In [ ]:
# Trực quan hóa ảnh hưởng của kích thước bầy đàn BPSO
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Biểu đồ 1: Chất lượng giải pháp vs Kích thước bầy đàn
axes[0].plot(df_swarm['swarm_size'], df_swarm['value'], marker='o', linewidth=2, markersize=8, color='#3498db')
axes[0].fill_between(df_swarm['swarm_size'], df_swarm['value'], alpha=0.3, color='#3498db')
axes[0].set_xlabel('Kích thước bầy đàn (Số lượng particles)', fontsize=12)
axes[0].set_ylabel('Tổng giá trị', fontsize=12)
axes[0].set_title('BPSO: Ảnh hưởng của kích thước bầy đàn lên chất lượng giải pháp', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].annotate(f'Tốt nhất: {df_swarm["value"].max():.0f}\n@{df_swarm.loc[df_swarm["value"].idxmax(), "swarm_size"]:.0f} particles', 
                 xy=(df_swarm.loc[df_swarm["value"].idxmax(), "swarm_size"], df_swarm["value"].max()),
                 xytext=(10, -30), textcoords='offset points', fontsize=10,
                 bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.7),
                 arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

# Biểu đồ 2: Chi phí tính toán vs Kích thước bầy đàn
axes[1].plot(df_swarm['swarm_size'], df_swarm['time'], marker='s', linewidth=2, markersize=8, color='#e74c3c')
axes[1].fill_between(df_swarm['swarm_size'], df_swarm['time'], alpha=0.3, color='#e74c3c')
axes[1].set_xlabel('Kích thước bầy đàn', fontsize=12)
axes[1].set_ylabel('Thời gian thực thi (giây)', fontsize=12)
axes[1].set_title('BPSO: Chi phí tính toán vs Kích thước bầy đàn', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Biểu đồ 3: Phân tích hiệu suất
axes[2].bar(range(len(df_swarm)), df_swarm['efficiency'], color='#9b59b6', alpha=0.7, edgecolor='black')
axes[2].set_xlabel('Kích thước bầy đàn', fontsize=12)
axes[2].set_ylabel('Hiệu suất (Giá trị/Thời gian)', fontsize=12)
axes[2].set_title('BPSO: Phân tích hiệu suất giải pháp', fontsize=13, fontweight='bold')
axes[2].set_xticks(range(len(df_swarm)))
axes[2].set_xticklabels(df_swarm['swarm_size'])
axes[2].grid(True, alpha=0.3, axis='y')
axes[2].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))

plt.tight_layout()
plt.show()

# Bảng tổng kết
print("\nCác chỉ số hiệu suất kích thước bầy đàn BPSO:")
print("="*80)
summary = df_swarm[['swarm_size', 'value', 'time', 'efficiency']].copy()
summary['time'] = summary['time'].apply(lambda x: f"{x:.6f}")
summary['efficiency'] = summary['efficiency'].apply(lambda x: f"{x:.2e}")
print(summary.to_string(index=False))

In [ ]:
### Nhận xét BPSO Iterations:

**Quan sát từ data thực tế:**
- Chất lượng giải pháp **không tăng tuyến tính** khi tăng Iterations:
  - 20 iterations: 81,328
  - 30 iterations: 83,455
  - **50 iterations: 78,787 (THẤP NHẤT!)** ⚠️
  - 70 iterations: 88,496
  - **100 iterations: 92,798 (CAO NHẤT!)** ✅
  - 150 iterations: 87,824 (giảm 5.4%)
- **⚠️ Vấn đề stability:**
  - Độ lệch chuẩn rất cao: 4,076 - 13,101 (5-16% so với value!)
  - Iterations = 20: std = 13,101 (16.1%) - **CỰC KỲ KHÔNG ỔN ĐỊNH**
  - Iterations = 150: std = 4,458 (5.1%) - tốt nhất nhưng value thấp
- Thời gian thực thi **tăng tuyến tính**: 0.009s → 0.046s (tăng 5x)

**Phân tích chi tiết:**
| Iterations | Value | Std Dev | Stability | Time | Nhận xét |
|------------|-------|---------|-----------|------|----------|
| 20 | 81,328 | 13,101 | ❌ Rất kém | 0.009s | Chưa hội tụ |
| 30 | 83,455 | 9,372 | ❌ Kém | 0.010s | Vẫn chưa đủ |
| 50 | 78,787 | 4,076 | ✅ Ổn định | 0.016s | **⚠️ BẪY NGẦM!** |
| 70 | 88,496 | 12,677 | ❌ Kém | 0.022s | Không ổn định |
| **100** | **92,798** | **9,097** | **⚠️ OK** | **0.031s** | **⭐ Best quality** |
| 150 | 87,824 | 4,458 | ✅ Ổn định | 0.046s | Overfitting? |

**⚠️ Hiện tượng bất thường tại Iterations = 50:**
- Value **thấp nhất** (78,787) trong khi stability lại **tốt nhất** (std = 4,076)
- **Nguyên nhân có thể:**
  - 50 iterations chưa đủ để explore solution space → **hội tụ sớm vào local optima**
  - Std thấp vì particles bị "mắc kẹt" ở cùng một vùng (premature convergence)
  - Đây là **false stability** - ổn định nhưng ở giải pháp kém

**Hiện tượng giảm chất lượng tại 150 iterations:**
- Value giảm từ 92,798 (100 iters) → 87,824 (150 iters) = **-5.4%**
- **Nguyên nhân:**
  - Có thể do over-exploration → particles "phá vỡ" giải pháp tốt đã tìm được
  - BPSO có exploration/exploitation trade-off - 150 iters có thể quá nhiều cho bài toán này
  - Random nature của BPSO - cần chạy nhiều lần để kết luận

**Kết luận:**
- **Iterations = 100** cho **chất lượng tốt nhất** (92,798)
- Iterations = 70 là **lựa chọn an toàn** nếu cần cân bằng (88,496, 0.022s)
- **TRÁNH Iterations = 50** - mặc dù stable nhưng quality kém (premature convergence)
- Iterations > 100: không cải thiện, thậm chí **giảm quality**

**So sánh với Swarm Size:**
- **Iterations ảnh hưởng mạnh hơn** đến chất lượng: range 78K-93K (±18%) vs 78K-91K (±17%)
- Thời gian tăng **chậm hơn**: 5x vs 10x (Swarm Size)
- **Ưu tiên tối ưu Iterations trước**, sau đó điều chỉnh Swarm Size
- **Khuyến nghị**: Iterations = 70-100, Swarm Size = 30-50

---

## 3.1.1.b: BPSO - Ảnh hưởng của kích thước bầy đàn

Phân tích ảnh hưởng của **Swarm Size** (số lượng particles) lên hiệu suất BPSO

In [ ]:
# Create comparison summary with efficiency calculation
print("="*100)
print(" "*35 + "PARAMETER ANALYSIS SUMMARY")
print("="*100)
print()

# GBFS Summary
print("🔹 GBFS (Greedy Best-First Search)")
print("-" * 100)
gbfs_best = df_gbfs.loc[df_gbfs['efficiency'].idxmax()]
print(f"   Best Configuration: Max States = {gbfs_best['max_states']:.0f}")
print(f"   ├─ Value:      {gbfs_best['value']:.2f}")
print(f"   ├─ Time:       {gbfs_best['time']:.6f} seconds")
print(f"   ├─ Efficiency: {gbfs_best['efficiency']:.2e}")
print(f"   └─ Std Dev:    {gbfs_best['value_std']:.2e} (extremely stable)")
print(f"   Key Finding:   Quality is PERFECT & STABLE across all Max States")
print()

# BPSO Swarm Size Summary
print("🔹 BPSO - Swarm Size Analysis")
print("-" * 100)
# Calculate efficiency for swarm
df_swarm['efficiency'] = df_swarm['value'] / df_swarm['time']
swarm_best_quality = df_swarm.loc[df_swarm['value'].idxmax()]
swarm_best_efficiency = df_swarm.loc[df_swarm['efficiency'].idxmax()]
print(f"   Best Quality:     Swarm Size = {swarm_best_quality['swarm_size']:.0f}")
print(f"   ├─ Value:         {swarm_best_quality['value']:.2f}")
print(f"   ├─ Time:          {swarm_best_quality['time']:.6f} seconds")
print(f"   └─ Std Dev:       {swarm_best_quality['value_std']:.2f}")
print(f"   Best Efficiency:  Swarm Size = {swarm_best_efficiency['swarm_size']:.0f}")
print(f"   ├─ Value:         {swarm_best_efficiency['value']:.2f}")
print(f"   ├─ Time:          {swarm_best_efficiency['time']:.6f} seconds")
print(f"   └─ Efficiency:    {swarm_best_efficiency['efficiency']:.2e}")
print(f"   Key Finding:   Size 30 gives BEST balance (quality vs speed)")
print()

# BPSO Iterations Summary
print("🔹 BPSO - Iterations Analysis")
print("-" * 100)
# Calculate efficiency for iterations
df_iter['efficiency'] = df_iter['value'] / df_iter['time']
iter_best_quality = df_iter.loc[df_iter['value'].idxmax()]
iter_best_efficiency = df_iter.loc[df_iter['efficiency'].idxmax()]
print(f"   Best Quality:     Iterations = {iter_best_quality['iterations']:.0f}")
print(f"   ├─ Value:         {iter_best_quality['value']:.2f}")
print(f"   ├─ Time:          {iter_best_quality['time']:.6f} seconds")
print(f"   └─ Std Dev:       {iter_best_quality['value_std']:.2f}")
print(f"   Best Efficiency:  Iterations = {iter_best_efficiency['iterations']:.0f}")
print(f"   ├─ Value:         {iter_best_efficiency['value']:.2f}")
print(f"   ├─ Time:          {iter_best_efficiency['time']:.6f} seconds")
print(f"   └─ Efficiency:    {iter_best_efficiency['efficiency']:.2e}")
print(f"   Key Finding:   100 iters for BEST quality, 70 for BEST efficiency")
print()

print("="*100)

# Comparative insights
print("\n📊 COMPARATIVE INSIGHTS:")
print("-" * 100)
print(f"   GBFS Max States Impact:    Value change = 0% (stable), Time improvement = {((df_gbfs.iloc[0]['time'] - df_gbfs.iloc[-1]['time']) / df_gbfs.iloc[0]['time'] * 100):.1f}%")
print(f"   BPSO Swarm Size Impact:    Value change = {((df_swarm['value'].max() - df_swarm['value'].min()) / df_swarm['value'].min() * 100):.1f}%, Time increase = {((df_swarm['time'].max() - df_swarm['time'].min()) / df_swarm['time'].min() * 100):.0f}%")
print(f"   BPSO Iterations Impact:    Value change = {((df_iter['value'].max() - df_iter['value'].min()) / df_iter['value'].min() * 100):.1f}%, Time increase = {((df_iter['time'].max() - df_iter['time'].min()) / df_iter['time'].min() * 100):.0f}%")
print("-" * 100)

### 🎯 Khuyến nghị cấu hình tham số (dựa trên data thực tế):

| Thuật toán | Tham số | Giá trị khuyến nghị | Lý do | Value đạt được |
|------------|---------|---------------------|-------|----------------|
| **GBFS** | Max States | **2500-5000** | Balance quality và time tốt nhất | 74K-84K |
| **BPSO** | Swarm Size | **30** | Cân bằng tốt nhất (quality cao, time OK) | 90,577 |
| **BPSO** | Swarm Size | **100** (nếu cần quality cao) | Quality cao nhất, chấp nhận chậm 3.4x | 91,222 |
| **BPSO** | Iterations | **100** | Quality cao nhất, đáng chờ | 92,798 |
| **BPSO** | Iterations | **70** (nếu cần nhanh) | Cân bằng tốt, tránh premature convergence | 88,496 |

**⚠️ Lưu ý:**
- **GBFS**: Max States càng cao → quality càng tốt (nhưng chậm hơn)
- **BPSO**: Độ lệch chuẩn cao (5-16%) → CẦN CHẠY NHIỀU LẦN và lấy trung bình
- **Cả hai** đều dùng fitness function: `alpha=0.7, beta=0.3`

### 📈 Trade-offs quan trọng:

**GBFS (TRUE Greedy Best-First Search):**
- ✅ **Ưu điểm**: 
  - **TRUE search algorithm** với priority queue, state tree, closed set
  - **Dùng cùng fitness function với BPSO** (alpha=0.7, beta=0.3) - đồng nhất!
  - **Deterministic** - kết quả có thể reproduce với cùng max_states
  - Nhanh hơn BPSO (~10-100x tùy max_states)
  - Max States parameter có ý nghĩa: giới hạn search space
  - Quality tăng khi tăng max_states (100→5000: 52K→84K, +60%)
- ⚠️ **Nhược điểm**: 
  - Greedy nature → có thể stuck ở local optima
  - Quality phụ thuộc max_states (trade-off time vs quality)
  - Exploration theo fitness gradient (không random như PSO)

**BPSO (Metaheuristic Optimization):**
- ✅ **Ưu điểm**: 
  - **Stochastic search** - có khả năng escape local optima
  - Không cần heuristic - tự khám phá solution space
  - Có thể xử lý bài toán phức tạp với nhiều ràng buộc
  - Linh hoạt, có thể tìm giải pháp "không trực quan"
- ⚠️ **Nhược điểm**: 
  - **Chậm hơn GBFS** (nhưng ít hơn so với old Greedy)
  - **CỰC KỲ KHÔNG ỔN ĐỊNH** (std: 4K-13K)
  - Cần tuning cẩn thận (Iterations, Swarm Size, w)
  - **Stochastic** - kết quả khác nhau mỗi lần chạy

### 🔬 Kết luận chính:

1. **✅ GBFS giờ đúng chuẩn lý thuyết:**
   - Priority queue (heapq) cho open set
   - Closed set để avoid revisiting
   - State expansion với nhiều child states
   - Max States parameter QUAN TRỌNG và có tác dụng

2. **✅ GBFS và BPSO dùng CÙNG fitness function:**
   - `fitness = 0.7 * revenue_norm + 0.3 * region_coverage_norm - penalty`
   - Đồng nhất về objective function
   - So sánh công bằng giữa 2 thuật toán

3. **Max States vs Iterations:**
   - GBFS max_states: Giới hạn số states explore (deterministic)
   - BPSO iterations: Số vòng lặp PSO (stochastic)
   - Cả hai đều trade-off giữa quality và time

4. **Khi nào dùng thuật toán nào?**
   
   **Dùng GBFS khi:**
   - Cần tốc độ và tính deterministic
   - Muốn control computational cost chính xác (max_states)
   - Bài toán có fitness landscape tương đối smooth
   - Cần kết quả ổn định, có thể reproduce
   
   **Dùng BPSO khi:**
   - Bài toán phức tạp, nhiều local optima
   - Có nhiều ràng buộc phức tạp
   - Chấp nhận trade-off: chậm hơn + ngẫu nhiên hơn
   - Cần explore solution space toàn diện
   - Sẵn sàng chạy nhiều lần để lấy kết quả tin cậy

5. **So sánh quality trên test case này:**
   - GBFS (max_states=5000): ~84K, deterministic
   - BPSO (best config): ~93K, nhưng std cao (±9K)
   - Cả hai đều sử dụng cùng multi-objective function
   - BPSO có potential cao hơn nhưng không ổn định